In [1]:
%load_ext lab_black

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait

chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
browser = webdriver.Chrome(
    executable_path="/Users/blakeduncan/Documents/chromedriver",
    chrome_options=chrome_options,
)

/Users/blakeduncan/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [3]:
import requests, csv, pandas as pd, pprint, time
from bs4 import BeautifulSoup
import lxml, html5lib
import re
from IPython.display import display, HTML

data_dict = {
    "name": [],
    "date": [],
    "platform": [],
    "score": [],
    "url": [],
    "ratings": [],
    "reviews": [],
    "userscore": [],
}  # Data Structure


def webpage(
    pageNum, system
):  # function that navigates the metacritic SRP(Search Results Pages) based on the page number
    url = (
        "https://www.metacritic.com/browse/games/score/metascore/90day/"
        + str(system)
        + "/filtered?view=condensed&page="
        + str(pageNum)
    )
    response = browser.get(url)
    wait = WebDriverWait(browser, 30)
    wait.until(
        lambda browser: browser.execute_script("return document.readyState")
        == "complete"
    )
    # userAgent = {"User-agent": "Mozilla/5.0"}
    # try:
    #     response = requests.get(url, headers=userAgent)
    # except requests.exceptions.RequestException as error:
    #     print(error)

    return response


def numberPages(
    response,
):  # Helper Function that determines how many pages are in a SRP to know how many times to run scrapper function
    # soup = BeautifulSoup(response.text, "html.parser")
    soup = BeautifulSoup(browser.page_source)
    pages = soup.find_all("li", {"class": "page last_page"})
    try:
        pagesCleaned = pages[0].find("a", {"class": "page_num"}).text
    except:
        pagesCleaned = "0"
    return pagesCleaned


def scrapper(num_loops, content):
    tblnum = 0
    while tblnum < num_loops:
        # get Game name
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for a in td[1].find_all("a", {"class": "title"}):
                data_dict["name"].append(a.find("h3").text)

        # get Game release date
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for date in td[1].find_all("span", {"class": ""}):
                data_dict["date"].append(date.text)

        # get platform
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for platform in td[1].find_all("span", {"class": "data"}):
                data_dict["platform"].append(platform.text.strip())

        # get Game score
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for user in td[0].find_all("div", {"class": "metascore_w"}):
                data_dict["score"].append(user.text.strip())

        # getting game url
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for a in td[1].find_all("a", {"class": "title"}, href=True):
                data_dict["url"].append(a["href"])
                # game_soup = BeautifulSoup(
                #     requests.get(
                #         "https://www.metacritic.com" + str(a["href"]) + "/user-reviews",
                #         headers={"User-agent": "Mozilla/5.0"},
                #     ).text,
                #     "html.parser",
                # )
                browser.get(
                    "https://www.metacritic.com" + str(a["href"]) + "/user-reviews"
                )
                wait = WebDriverWait(browser, 30)
                wait.until(
                    lambda browser: browser.execute_script("return document.readyState")
                    == "complete"
                )
                game_soup = BeautifulSoup(browser.page_source)
                game_ratings = game_soup.find("strong", text=re.compile("Ratings"))
                if game_ratings:
                    game_ratings = (
                        game_ratings.get_text().strip().replace(" Ratings", "")
                    )
                else:
                    game_ratings = "0"
                data_dict["ratings"].append(game_ratings)

                # critic_soup = BeautifulSoup(
                #     requests.get(
                #         "https://www.metacritic.com"
                #         + str(a["href"])
                #         + "/critic-reviews",
                #         headers={"User-agent": "Mozilla/5.0"},
                #     ).text,
                #     "html.parser",
                # )
                browser.get(
                    "https://www.metacritic.com" + str(a["href"]) + "/critic-reviews"
                )
                wait = WebDriverWait(browser, 30)
                wait.until(
                    lambda browser: browser.execute_script("return document.readyState")
                    == "complete"
                )
                critic_soup = BeautifulSoup(browser.page_source)
                critic_reviews = critic_soup.find(
                    "strong", text=re.compile("Critic Reviews")
                )
                if critic_reviews:
                    critic_reviews = (
                        critic_reviews.get_text().strip().replace(" Critic Reviews", "")
                    )
                else:
                    critic_reviews = "0"
                data_dict["reviews"].append(critic_reviews)

        # get Game userscore
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for score in td[1].find_all("div", {"class": "metascore_w"}):
                data_dict["userscore"].append(score.text)
        tblnum += 1


def pages(
    lastPageNum, system
):  # Function that returns the html(code) and initiates the web scrapper
    currentPage = 0
    while currentPage < int(lastPageNum):
        url = url = (
            "https://www.metacritic.com/browse/games/score/metascore/90day/"
            + str(system)
            + "/filtered?view=condensed&page="
            + str(currentPage)
        )
        userAgent = {"User-agent": "Mozilla/5.0"}
        response = requests.get(url, headers=userAgent)
        soup = BeautifulSoup(response.text, "html.parser")
        content = soup.find_all("table")

        num_loops = len(content)
        # print(num_loops)
        scrapper(num_loops, content)
        # print(data_dict)
        currentPage += 1
        time.sleep(6)


def main():
    # systems = ["switch", "ps4", "ps5"]
    systems = ["all"]
    for system in systems:
        numPage = numberPages(webpage(0, system))
        pages(int(numPage), system)
        time.sleep(5)
    xData = pd.DataFrame.from_dict(data_dict)
    xData.to_csv("mc_90_days.csv")
    browser.close()


main()

In [4]:
video_games = pd.DataFrame.from_dict(data_dict)

video_games = video_games[~video_games["userscore"].isin(["tbd"])]
video_games = video_games[~video_games["score"].isin(["tbd"])]
video_games = video_games[~video_games["ratings"].isin(["tbd"])]
video_games = video_games[~video_games["reviews"].isin(["tbd"])]

video_games["ratings"] = video_games["ratings"].astype(float)
video_games["reviews"] = video_games["reviews"].astype(float)
video_games["score"] = video_games["score"].astype(float) * video_games["reviews"]
video_games["userscore"] = (
    video_games["userscore"].astype(float) * video_games["ratings"]
)

d = {
    "date": "date",
    "platform": "platform",
    "score": "score",
    "ratings": "ratings",
    "userscore": "userscore",
}
video_games = (
    video_games.groupby("name")
    .agg(
        {
            "date": "min",
            "platform": list,
            "score": "sum",
            "ratings": "sum",
            "reviews": "sum",
            "userscore": "sum",
        }
    )
    .rename(columns=d)
)

video_games["userscore"] = (video_games["userscore"] / video_games["ratings"]).round(1)
video_games["score"] = (video_games["score"] / video_games["reviews"]).round(0)
video_games["ratings"] = video_games["ratings"].astype(int)
video_games["reviews"] = video_games["reviews"].astype(int)
video_games["score"] = video_games["score"].fillna(0).astype(int)

video_games = video_games[
    ((video_games["userscore"] * 10) + video_games["score"]) / 2 >= 75
]
video_games = video_games[video_games["ratings"] >= 100]

display(HTML(video_games.sort_values(["ratings"], ascending=False).to_html()))

,date,platform,score,ratings,reviews,userscore
name,,,,,,
Resident Evil 4,"March 24, 2023","[PlayStation 5, PC, Xbox Series X]",93,5589,166,5.7
Hogwarts Legacy,"February 7, 2023","[Xbox Series X, PlayStation 5, PC]",84,5255,135,8.6
Hi-Fi Rush,"January 25, 2023","[PC, Xbox Series X]",88,3709,89,8.9
Dead Space,"January 27, 2023","[Xbox Series X, PlayStation 5, PC]",89,1579,143,8.8
Wo Long: Fallen Dynasty,"March 3, 2023","[PlayStation 5, PC, Xbox Series X]",80,1314,128,8.1
Wild Hearts,"February 17, 2023","[PlayStation 5, Xbox Series X, PC]",77,1223,116,8.5
Fatal Frame: Mask of the Lunar Eclipse,"March 9, 2023","[Switch, PC, PlayStation 5, Xbox Series X]",68,1013,65,8.4
Metroid Prime Remastered,"February 8, 2023",[Switch],94,817,92,8.8
Octopath Traveler II,"February 24, 2023","[PlayStation 5, Switch, PC]",85,242,97,7.5
